## Testa den omtränade modellen på alla testbilder
**Mål:**
- Ladda den omtränade modellen och köra den på testbilder
- Se felklassificerade bilder
- Analysera resultaten

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models  # Importera models direkt
from PIL import Image
import os
import shutil
import numpy as np
import psutil

# Kolla batteristatus
battery = psutil.sensors_battery()
if battery and battery.power_plugged is False:
    print("⚠️ Warning: Your laptop is running on battery. Performance may be reduced, "
          "and CUDA might fail due to power-saving features.")

# Check CUDA availability
if torch.cuda.is_available():
    print(f"✅ CUDA is available: Running on {torch.cuda.get_device_name(0)}")
else:
    print("❌ CUDA is not available: Running on CPU. Ensure your GPU drivers are installed correctly.")

# Ladda modellen och sätt i eval-mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🛠 KORRIGERA VARNINGAR genom att använda "weights"
model = models.mobilenet_v2(weights=None)  # Istället för pretrained=False

# Anpassa klassificeraren
model.classifier[1] = torch.nn.Linear(model.last_channel, 2)

# Ladda tränade vikter
model.load_state_dict(torch.load('fire_classifier_retrained.pth', map_location=device))
model.to(device)
model.eval()

# Definiera testmapp
test_folders = {
    'fire': 'dataset/fire_images',
    'non_fire': 'dataset/non_fire_images'
}

# ✅ Definiera samma transform som vid träning
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Anpassa storleken till MobileNetV2s förväntade input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisering enligt MobileNetV2
])

print("✅ Modell laddad och redo för testning!")


## Gå igenom alla brandbilder och inte brandbilder

In [ ]:
true_labels = []
pred_labels = []
misclassified = []

# Kör inferens på alla testbilder
for label, folder in test_folders.items():
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)

        # Ladda & preprocessa bilden
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        # Gör en prediktion
        with torch.no_grad():
            output = model(image)
            probabilities = F.softmax(output, dim=1)
            confidence, predicted_class = torch.max(probabilities, 1)

        # Konvertera etiketter till numeriskt format för confusion matrix
        label_numeric = 1 if label == 'fire' else 0
        predicted_numeric = predicted_class.item()

        # Lagra resultat
        true_labels.append(label_numeric)
        pred_labels.append(predicted_numeric)

        # Om felklassificerad, spara i lista
        if predicted_numeric != label_numeric:
            predicted_label = ""
            if predicted_numeric == 1:
                predicted_label = "fire"
            else:
                predicted_label = "non_fire"
            misclassified.append((img_path, label, 'fire' if predicted_numeric == 1 else 'non_fire', confidence.item() * 100))
            print(f'❌ {img_path} | True: {label} | Pred: {predicted_label} ({confidence.item() * 10:.2f}%)')
print(f"✅ Testning klar! Totalt {len(true_labels)} bilder analyserade.")
print(f"❌ Felklassificerade bilder: {len(misclassified)}")


## 📊 Visa Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Skapa confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
class_names = ['non_fire', 'fire']

# Beräkna accuracy
accuracy = (cm[0, 0] + cm[1, 1]) / cm.sum() * 100

# Plotta confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title(f"Confusion Matrix: Test Set\nAccuracy: {accuracy:.2f}%")  # Lägg till accuracy i titeln
plt.show()

## ❌ Visa Felklassificerade Bilder
**Vad gick fel?**
- Visa några bilder som modellen klassificerade fel
- Jämför dem med en **lokal LLM (t.ex. Llama 3.2 Vision)**

In [ ]:
from IPython.display import display

# Visa de 20 första felklassificerade bilderna
for img_path, true_label, predicted_label, confidence in misclassified[:20]:
    print(f'❌ {img_path} | True: {true_label} | Pred: {predicted_label} ({confidence:.2f}%)')
    display(Image.open(img_path))

## 🤖 Llama 3.2 Vision analysera samma bilder
Kan en **LLM förstå bilden bättre** än vår specialtränade AI?

In [ ]:
import ollama
import textwrap

# Testa en bild med Llama 3.2 Vision
llama_image = misclassified[0][0]  # Första felklassificerade bilden

response = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'What is in this image? Please provide reasoning step by step.',
        'images': [llama_image]
    }]
)

# Formatera outputen
wrapped_response = textwrap.fill(response['message']['content'], width=80)

print(f'📸 Bild: {llama_image}')
display(Image.open(llama_image))
print('\n🧠 Llama 3.2 Vision Response:\n')
print(wrapped_response)